## Resegmentation of grid manager

In [ ]:
import os
from pathlib import Path
from grid_manager import GridManager
import math

data_dir = os.path.join("grids", "with-is-residential")
target_dir = os.path.join("grids", "with-is-residential-resegmented")

target_segment_h = 500
target_segment_w = 500

folder = Path(data_dir)
targets = [(f.name, GridManager(f.name, data_dir=data_dir)) for f in folder.iterdir() if f.is_file()]
print(f"files: {[filename for filename, manager in targets]}")

In [ ]:
for filename, grid_manager in targets:

    print(f"resegmenting file: {filename}: ", end="")

    metadata = grid_manager.get_metadata()

    result = GridManager(
        filename,
        metadata.rows_number,
        metadata.columns_number,
        metadata.upper_left_longitude,
        metadata.upper_left_latitude,
        metadata.grid_density,
        target_segment_h,
        target_segment_w,
        data_dir=target_dir,
        third_dimension_size=metadata.third_dimension_size
    )

    metadata = grid_manager.get_metadata()

    fragment_row, fragment_col = 2500, 2500
    fragment_height, fragment_width = 1500, 1500

    counter = 0

    for segment_row in range(math.ceil(metadata.rows_number / metadata.segment_h)):
        for segment_column in range(math.ceil(metadata.columns_number / metadata.segment_w)):
            segment_start_row = segment_row * metadata.segment_h
            segment_end_row = min(
                (segment_row + 1) * metadata.segment_h,
                metadata.rows_number
            )
            segment_start_column = segment_column * metadata.segment_w
            segment_end_column = min(
                (segment_column + 1) * metadata.segment_w,
                metadata.columns_number
            )
            copied_h = segment_end_row - segment_start_row
            copied_w = segment_end_column - segment_start_column

            result.write_arbitrary_fragment(
                grid_manager.read_arbitrary_fragment(
                    segment_row * metadata.segment_h,
                    segment_column * metadata.segment_w,
                    copied_h,
                    copied_w,
                ),
                segment_row * metadata.segment_h,
                segment_column * metadata.segment_w
            )
            print(f"{counter}|", end="")
            counter += 1
    print()

## Verification

In [ ]:
from grid_manager import GridManager
import matplotlib.pyplot as plt
import cv2
import numpy as np
import time
from trainer.model import PREDICT_GRID_INDICES


grid_manager = GridManager("Chorzów-Polska.city_grid", data_dir="grids/test/with-is-residential-resegmented")
metadata = grid_manager.get_metadata()

assert metadata.third_dimension_size == 3, "You have an outdated version of GridManager. Consider downloading grid files again to get all data..."

h, w = metadata.rows_number, metadata.columns_number
fragment_height, fragment_width = 2000, 2000
fragment_row, fragment_col = h//2 - fragment_height//2, w//2 - fragment_width//2


img = grid_manager.read_arbitrary_fragment(
    fragment_row, fragment_col,
    fragment_height,
    fragment_width
)[:, :, PREDICT_GRID_INDICES.IS_STREET]

print(f"DEBUG: img.max(): {img.max()}")

struct_el = np.ones((3,3))
dilated = cv2.dilate(img, struct_el, iterations=3)

fig, axs = plt.subplots(1, 2)
plt.gray()
axs[0].set_title("original")
axs[0].imshow(img)

axs[1].set_title("dilated")
axs[1].imshow(dilated)